# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data = pd.read_csv("../output_data/cities.csv")
output_data

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tasiilaq,65.613456,-37.633570,19.40,73,75,26.46,GL,1613097454
1,1,bethel,37.251852,-95.231483,17.60,73,90,16.11,US,1613097454
2,2,taolanaro,-25.022531,46.985369,75.20,100,75,9.22,MG,1613097454
3,3,itarema,-3.031859,-39.857912,78.13,83,100,9.17,BR,1613097454
4,4,bambous virieux,-20.343862,57.763682,78.80,73,75,8.05,MU,1613097455
...,...,...,...,...,...,...,...,...,...,...
619,625,sundumbili,-29.133038,31.379892,69.80,94,40,2.30,ZA,1613097529
620,626,okhotsk,59.358460,143.203491,18.46,90,95,7.94,RU,1613097530
621,627,wagar,42.038509,-83.378946,19.99,67,90,4.61,US,1613097530
622,628,mahaicony,6.573360,-57.809786,73.40,94,0,4.61,GY,1613097530


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = output_data[["Latitude", "Longitude"]].astype(float)
humidity = output_data['Humidity'].astype(float)

In [5]:
fig = gmaps.figure(center=(10,20), zoom_level=.7)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_output = output_data.loc[((output_data['Max Temp'] > 65) & 
                                (output_data['Max Temp'] < 75) &
                                (output_data['Humidity'] < 25))]
ideal_output

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
101,101,sangamner,19.576117,74.207019,71.47,23,63,4.03,IN,1613097467
159,160,twentynine palms,34.135558,-116.054169,66.99,24,40,14.97,US,1613097474
216,217,gogrial,8.532399,28.099100,69.69,22,0,5.59,SS,1613097481
291,293,araouane,18.904265,-3.527516,69.62,17,47,10.83,ML,1613097490
363,366,tessalit,20.257353,0.991030,65.05,21,0,6.24,ML,1613097499
475,481,magaria,13.003466,8.907978,66.56,23,0,8.95,NE,1613097512
493,499,nioro,15.232384,-9.590751,73.81,18,49,7.87,ML,1613097514


In [7]:
ideal_locations = ideal_output[["Latitude", "Longitude"]].astype(float)
ideal_humidity = ideal_output['Humidity'].astype(float)

In [8]:
new_fig = gmaps.figure(center=(10,20), zoom_level=.7)

ideal_heat_layer = gmaps.heatmap_layer(ideal_locations, weights=ideal_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

new_fig.add_layer(ideal_heat_layer)

new_fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
ideal_output['Hotel Name'] = ""
ideal_output

C:\Users\16129\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
101,101,sangamner,19.576117,74.207019,71.47,23,63,4.03,IN,1613097467,
159,160,twentynine palms,34.135558,-116.054169,66.99,24,40,14.97,US,1613097474,
216,217,gogrial,8.532399,28.099100,69.69,22,0,5.59,SS,1613097481,
291,293,araouane,18.904265,-3.527516,69.62,17,47,10.83,ML,1613097490,
363,366,tessalit,20.257353,0.991030,65.05,21,0,6.24,ML,1613097499,
475,481,magaria,13.003466,8.907978,66.56,23,0,8.95,NE,1613097512,
493,499,nioro,15.232384,-9.590751,73.81,18,49,7.87,ML,1613097514,


In [10]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_output.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row['City Name']
    country = row['Country']

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#    print(name_address.url)

    # convert to json
    hotel = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ideal_output.loc[index, "Hotel Name"] = hotel['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
ideal_output

C:\Users\16129\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
101,101,sangamner,19.576117,74.207019,71.47,23,63,4.03,IN,1613097467,Malpani Health Club & Resort
159,160,twentynine palms,34.135558,-116.054169,66.99,24,40,14.97,US,1613097474,Campbell House
216,217,gogrial,8.532399,28.099100,69.69,22,0,5.59,SS,1613097481,
291,293,araouane,18.904265,-3.527516,69.62,17,47,10.83,ML,1613097490,
363,366,tessalit,20.257353,0.991030,65.05,21,0,6.24,ML,1613097499,
475,481,magaria,13.003466,8.907978,66.56,23,0,8.95,NE,1613097512,
493,499,nioro,15.232384,-9.590751,73.81,18,49,7.87,ML,1613097514,Hotel Guetema City


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_output.iterrows()]
locations = ideal_output[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
hotel_symbol_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)', 
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5, 
    info_box_content=[f"{hotel}" for hotel in hotel_info])


hotel_fig = gmaps.figure()
hotel_fig.add_layer(hotel_symbol_layer)

hotel_fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Display Figure
combined_fig = gmaps.figure()

combined_fig.add_layer(ideal_heat_layer)
combined_fig.add_layer(hotel_symbol_layer)

combined_fig


Figure(layout=FigureLayout(height='420px'))